### Assignment 2

This is Assignment 2. Deadline for submission is November, 1, 11-59pm EST. Please send your solutions in a gzipped archive to dmm2017@med.cornell.edu, with copy to chb4004@med.cornell.edu. No extensions will be granted. However, if you want to check preliminary correctness of your solution, you can send your txt files with answers on October, 27. On October 28, you will receive a preliminary grading of your solutions, so you can improve them needed. 

Your archive should contain this ipynb file with the code you wrote. However, if you are not comfortable with python, you can write in any other programming language, but provide instructions how to run/compile your code. Also, refer to the "Output data" section.

## BWT-aligner

The goal of this project is to build a very basic read mini-aligner and use it to find SNPs in the genome.

Everything is simple. You will be given a reference genome. We sampled some 100bp single reads from it (this process resembles sequencing). We want to know the positions of the reads on the genome. In order to make things a little bit harder, we add few SNPs to the reference genome, but there are no reads that cover more that one SNP. Basically, it means that the distance between neighbouring SNPs is large. There are no sequencing errors and any other events. Finally, reads are sampled from the forward strand only (no need to do reverse complement index).

Your task is to find alignments of reads to the reference genome. You need to report only alignment positions for the first nucleotide of each read. Also you will need to report SNPs that you have found in the genome.

## Approximate solution

Your solution should contain:

0. Input parsing - you can use whatever you want (I would propose Biopython as a state-of-the-art package).

1. Suffix array/BWT construction - you may use suffix array from any package / just copy the code and make a reference. But you should undertand what is the time complexity of your suffix array construction (shouldn't be more than O(nlogn)). Then write your own code to compute BWT transormation and FM-index construction.

2. Write basic BWT backward search and check how many reads are aligned.

3. Add some embellishments to map reads that cover SNPs.

4. Using your mappings, find postions with SNPs on the reference genome.

5. Prepare a short report on your solution. What are the key steps? What if read have some sequencing errors? How could it affect your current solution? 

## Input data

You will be provided a small sample with a known answer, and a test sample with an unknown answer.
Small sample has the following files:

- sample_genome.fasta - a small genome
- sample_reads.fasta - file with 100 simulated reads
- sample_read_positions.txt - file with read start positions (0-indexed)  
- sample_snps.txt - file with SNPs positions and true nucleotides.

Test sample:
- test_genome.fasta - medium-sized genome
- test_reads.fasta - file with 10000 simulated reads




## Output data

Your output files are:

- test_read_positions.txt - file with read start positions (0-indexed). I-th line in this file should be a position for i-th read in reads.fasta. If you can't find the position for the read, write -1 as your answer.
- test_snps.txt - file with snp positions (0-indexed) and true nucleotides. Check snps.txt from the small sample for format.

## Grading

- 30% - correct read positions
- 10% - correct SNP positions and nucleotides
- 30% - reasoning/algorithm description
- 30% - code quality

In [ ]:
import pandas as pd
from Bio import SeqIO
import numpy as np
from pysuffixarray.core import SuffixArray
from collections import Counter

#Collaborated with Yohyoh Wang 

def parse_fasta(file_path):
	with open(file_path) as fasta:
		fasta_seqs = [str(i.seq) for i in SeqIO.parse(fasta, "fasta")]
		if len(fasta_seqs) == 1:
			return fasta_seqs[0]
		elif len(fasta_seqs) > 1:
			return fasta_seqs

ref_genome = parse_fasta('‘test_sample/test_genome.fasta')
#print(ref_genome)
reads = open_fasta('‘test_sample/test_reada.fasta')


'''I used this package to construct a suffix array https://pypi.org/project/pysuffixarray/'''
sa = SuffixArray(ref_genome) #constructs a suffix array
ref_sa = sa.suffix_array() #returns a suffix array

genome = ref_genome + '$' + ref_genome

#Returns BWT of a given string
def BWT(t):
	#creates a list of all cyclic rotations of t
	rotation = [t[i:]+t[:i] for i in range(len(t))]
	#sort the suffixes
	rotation.sort()
	#joining the last character of each suffix, which is the BWT
	return "".join(r[-1] for r in rotation)

#Returns BWT from suffix array
def BWT_SA(t, sa):
	return "".join(t[i-1] for i in sa)

#Returns the last and first column elements of the BW Matrix and the index of each
def LF(seq, s_a):
	#returns last column in BW matrix (bwt)
	L = [seq[i-1] for i in s_a]
	#returns first column in BW Matrix
	F = [seq[i] for i in s_a]
	return L, F

#Returns the FM Index or ranks of each nucleotide in the FM Index
def FM_index(f, l):
	count_f = {}
	count_l = {}
	ranks_f = []
	ranks_l = []
	for c in f:
		if c not in count_f:
			count_f[c] = 1
		ranks_f.append(c + str(count_f[c]))
		count_f[c] += 1
	for d in l:
		if d not in count_l:
			count_l[d] = 1
		ranks_l.append(d + str(count_l[d]))
		count_l[d] += 1
	return ranks_f, ranks_l

#Performs the alignment and counts the number of SNPs
def BWA(reads, f, l, ranks_f, ranks_l, sa):
	all_pos = []
	snps = {}
	j = 0
	while j < len(reads)-1:
		read = reads[j]
		for i in range(1, len(read)):

			if i == 1:
				trace_F = [k for k, v in enumerate(f) if read[-i] in f[k]]
				trace_L = [k for k in trace_F if read[-i-1] in l[k]]
			else:
				trace_F = [ranks_f.index(ranks_l[k]) for k in trace_L]
				trace_L = [k for k in trace_F if read[-i-1] in l[k]]

			if len(trace_L) == 1: #Calculating read positions if there is an alignment
				end_pos = trace_L[0]
				start_pos = sa[end_pos]+i-len(read)
			else:
				if i > 15:
					all_pos[j] = start_pos[r]
					if i < len(read)-1: #this indicates snp
						snps[sa[end_pos]-2] = read[-i-1]
					j = j + 1
					break
				else:
					reads[j] = read[0:-15]
					j = j - 1
					break
	return all_pos, snps

l, f = LF(genome, ref_sa)
first_ranks, last_ranks = FM_index(f, l)
final_pos, snp = BWA(reads, f, l, first_ranks, last_ranks, ref_sa)
print(final_pos)
print(snp)

textfile = open("final_pos.txt", "w")
for element in final_pos:
    textfile.write(element + "\n")
textfile.close()

My Key Steps were: 
1. Construct Suffix Array 
2. Compute BWT from Suffix Array 
3. Compute FM Index 
4. Use LF mapping to align the strings. It's basically a string matching problem, where all strings (Reads) need to be
mapped to the reference genome.
5. My logic to find the SNPs was it is unlikely that if there is no alignment, that there would be a match of 15 base pairs. In this case, 
it is very likely that we encountered a SNP. 

It's very likely that realistic reads will have sequencing errors. This can be due to normal genetic variation or mutations. On average, there’s a difference of 1 in 1000 characters between the DNA sequence of two people. Matching of the reads could be important, but we could account for some mismatch. My current solution works purely based on exact matching. My proposed solution for accounting for mismatches would be to use a band technique. If we are able to do backward search by identifying a "band" on the BW matrix, this could solve such a problem. We could identify a "band" that's an exact match and other bands that account for mismatches. 